<a href="https://colab.research.google.com/github/narayana142002/ChatBot_for_Hapkey/blob/main/LLModel_training_for_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install llama-index
!pip install llama-index-embeddings-huggingface
!pip install peft
!pip install auto-gptq
!pip install optimum
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 94.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.5/362.5 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfull

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

### Define Settings

In [ ]:
# import any embedding model on HF hub (https://huggingface.co/spaces/mteb/leaderboard)
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
# Settings.embed_model = HuggingFaceEmbedding(model_name="thenlper/gte-large") # alternative model

Settings.llm = None
Settings.chunk_size = 256
Settings.chunk_overlap = 25

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

LLM is explicitly disabled. Using MockLLM.


### Read and Store Docs into Vector DB

In [ ]:
file_path="/content/canto_1.txt"

In [ ]:
# prompt: read the text file

documents = SimpleDirectoryReader(input_files=[file_path]).load_data()


In [ ]:
# prompt: do this for the document and put in the document back the updated content

# Normalize text by removing special characters, if needed
import unicodedata
new_documents = []
for doc in documents:
  # Remove diacritics (accents) from characters
  normalized_text = unicodedata.normalize('NFKD', doc.text).encode('ASCII', 'ignore').decode()

  # Optional: Simplify text further by lowering the case
  simplified_text = normalized_text.lower()
  doc.text = simplified_text
  new_documents.append(doc)

index = VectorStoreIndex.from_documents(new_documents)


In [ ]:
new_documents[0].text

'\r\nchapter one\r\nquestions by the sages\r\ntext 1\r\ntranslation\r\no my lord, sri krsna, son of vasudeva, o all-pervading personality of godhead, i offer my respectful obeisances unto you. i meditate upon lord sri krsna because he is the absolute truth and the primeval cause of all causes of the creation, sustenance and destruction of the manifested universes. he is directly and indirectly conscious of all manifestations, and he is independent because there is no other cause beyond him. it is he only who first imparted the vedic knowledge unto the heart of brahmaji, the original living being. by him even the great sages and demigods are placed into illusion, as one is bewildered by the illusory representations of water seen in fire, or land seen on water. only because of him do the material universes, temporarily manifested by the reactions of the three modes of nature, appear factual, although they are unreal. i therefore meditate upon him, lord sri krsna, who is eternally existen

In [ ]:

# some ad hoc document refinement
print(len(documents))
for doc in documents:
    if "Member-only story" in doc.text:
        documents.remove(doc)
        continue

    if "The Data Entrepreneurs" in doc.text:
        documents.remove(doc)

    if " min read" in doc.text:
        documents.remove(doc)

print(len(documents))

SyntaxError: invalid syntax (<ipython-input-45-04bcd1f66582>, line 1)

### Set Up Search Function

In [ ]:
# set number of docs to retreive
top_k = 3

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=top_k,
)

In [ ]:
# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.5)],
)

### Retrieve Relevant Docs

In [ ]:
# prompt: WE CAN ASK MULTIPLE QUESTIONS AND THEY NEED TO ANSWER LIKE A CHAT BOT

while True:
  query = input("What do you want to ask? ")
  if query == "exit":
    break
  response = query_engine.query(query)
  print(response)


In [ ]:
# query documents
query = "wHO IS GOD?"
response = query_engine.query(query)

In [ ]:
# reformat response
context = "Context:\n"
for i in range(top_k):
    context = context + response.source_nodes[i].text + "\n\n"

print(context)

Context:
no one is therefore either an enemy or friend of the lord. everyone is suffering and enjoying the result of his own destiny. this destiny is made by the living beings in course of social intercourse. everyone here wants to lord it over the material nature, and thus everyone creates his own destiny under the supervision of the supreme lord. he is all-pervading and therefore he can see everyones activities. and because the lord has no beginning or end, he is known also as the eternal time, kala.

text 29
translation
o lord, no one can understand your transcendental pastimes, which appear to be human and so are misleading. you have no specific object of favor, nor do you have any object of envy. people only imagine that you are partial.

purport
the lords mercy upon the fallen souls is equally distributed. he has no one as the specific object of hostility. the very conception of the personality of godhead as a human being is misleading. his pastimes appear to be exactly like a hu

### Import LLM

In [ ]:
# load fine-tuned model from hub
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

config = PeftConfig.from_pretrained("shawhin/shawgpt-ft")
model = PeftModel.from_pretrained(model, "shawhin/shawgpt-ft")

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

config.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4565: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
Some weights of the model checkpoint at TheBloke/Mistral-7B-Instruct-v0.2-GPTQ were not used when initializing MistralForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.la

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/8.40M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

## Fine tunning the LLM

In [ ]:
!pip install transformers datasets torch

In [ ]:
import pandas as pd

# Load the text data
with open('/content/canto_1.txt', 'r', encoding='utf-8') as file:
    text = file.read()

# Optional: Split the text into chunks (e.g., if it’s too large)
chunks = text.split('\n\n')  # Split by paragraphs or chapters


In [ ]:
import unicodedata

# Function to normalize and clean text
def clean_text(chunk):
    # Normalize text (remove accents, etc.)
    normalized_text = unicodedata.normalize('NFKD', chunk).encode('ASCII', 'ignore').decode()
    # Optional: Lowercase the text
    return normalized_text.lower()

# Apply the cleaning function to each chunk
cleaned_chunks = [clean_text(chunk) for chunk in chunks]

chunks = cleaned_chunks


In [ ]:
from transformers import AutoTokenizer

# Load a pre-trained tokenizer
tokenizer = AutoTokenizer.from_pretrained('gpt2')

# Option 1: Use the eos_token as pad_token
tokenizer.pad_token = tokenizer.eos_token

# OR Option 2: Add a new pad token
# tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Tokenize the text
tokenized_data = tokenizer(chunks, padding=True, truncation=True, return_tensors='pt', max_length=512)


In [ ]:
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer

# Assuming tokenized_data is a dictionary with 'input_ids' and 'attention_mask'
dataset = Dataset.from_dict(tokenized_data)
def shift_labels(batch):
    labels = batch['input_ids'].copy()
    labels[:-1] = labels[1:]
    labels[-1] = -100
    batch['labels'] = labels
    return batch

dataset = dataset.map(shift_labels)

model_name = "gpt2"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    num_train_epochs=1,
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,)
trainer.train()

Map:   0%|          | 0/1968 [00:00<?, ? examples/s]

Step,Training Loss


TrainOutput(global_step=492, training_loss=1.6706965996967098, metrics={'train_runtime': 311.7189, 'train_samples_per_second': 6.313, 'train_steps_per_second': 1.578, 'total_flos': 514222718976000.0, 'train_loss': 1.6706965996967098, 'epoch': 1.0})

In [ ]:
# from torch.utils.data import Dataset

# class BhagavatamDataset(Dataset):  # Inherit from torch.utils.data.Dataset
#     def __init__(self, tokenized_data):
#         self.input_ids = tokenized_data['input_ids']
#         self.attention_mask = tokenized_data['attention_mask']

#     def __len__(self):
#         return len(self.input_ids)

#     def __getitem__(self, idx):
#         input_id = self.input_ids[idx]
#         attention_mask = self.attention_mask[idx]

#         # Create labels by shifting the input IDs
#         labels = input_id.clone()  # Make a copy of the input IDs
#         labels[:-1] = input_id[1:]  # Shift left
#         labels[-1] = -100  # Set the last label to -100 (ignored during loss calculation)

#         return {
#             'input_ids': input_id,
#             'attention_mask': attention_mask,
#             'labels': labels  # Include the labels
#         }


In [ ]:
# Trainer()

In [ ]:
# # Load the model
# model = AutoModelForCausalLM.from_pretrained('gpt2')

# # Define training arguments
# training_args = TrainingArguments(
#     output_dir='./results',
#     per_device_train_batch_size=2,
#     num_train_epochs=3,
#     save_steps=10_000,
#     save_total_limit=2,
#     logging_dir='./logs',
# )

# # Create a Trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=final_dataset,
# )

# # Start training
# trainer.train()

In [ ]:
model.save_pretrained('./bhagavatam_model')
tokenizer.save_pretrained('./bhagavatam_model')


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 50, 'do_sample': True}


('./bhagavatam_model/tokenizer_config.json',
 './bhagavatam_model/special_tokens_map.json',
 './bhagavatam_model/vocab.json',
 './bhagavatam_model/merges.txt',
 './bhagavatam_model/added_tokens.json',
 './bhagavatam_model/tokenizer.json')

In [ ]:
import pickle
import os

# Path to the saved model directory
model_dir = './bhagavatam_model'

# Create a dictionary to hold model files
model_data = {}

# Load each file from the directory into the dictionary
for filename in os.listdir(model_dir):
    with open(os.path.join(model_dir, filename), 'rb') as f:
        model_data[filename] = f.read()

# Save the entire dictionary as a pickle file
with open('bhagavatam_model.pkl', 'wb') as pkl_file:
    pickle.dump(model_data, pkl_file)


In [ ]:
# prompt: download the folder named Bhagavatam model from contents

!zip -r /content/bhagavatam_model.zip /content/bhagavatam_model
from google.colab import files
files.download("/content/bhagavatam_model.zip")


  adding: content/bhagavatam_model/ (stored 0%)
  adding: content/bhagavatam_model/generation_config.json (deflated 31%)
  adding: content/bhagavatam_model/model.safetensors (deflated 7%)
  adding: content/bhagavatam_model/config.json (deflated 52%)
  adding: content/bhagavatam_model/tokenizer_config.json (deflated 51%)
  adding: content/bhagavatam_model/tokenizer.json (deflated 72%)
  adding: content/bhagavatam_model/special_tokens_map.json (deflated 74%)
  adding: content/bhagavatam_model/vocab.json (deflated 59%)
  adding: content/bhagavatam_model/merges.txt (deflated 53%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# prompt: save the bhagavatam_model folder into my google drive

from google.colab import drive
drive.mount('/content/drive')

!cp -r /content/bhagavatam_model /content/drive/MyDrive/


Mounted at /content/drive


In [ ]:
# prompt: how to pass the input to the above model and get response from the gpt-2

from transformers import pipeline

# Load the fine-tuned model and tokenizer
model_path = './bhagavatam_model'
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Create a text generation pipeline

generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

# Define your input text
input_text = "Tell me about the story of King Parikshit"

# Generate text
result = generator(input_text, max_length=100, num_return_sequences=1)

# Print the generated text
print(result[0]['generated_text'])


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Tell me about the story of King Parikshit A���i. He a by for king Mahprhu in Kura, was the of great and were great, so was aig��ī the. not of kingā He to the toig�r but the.ī that emperor to king He king his. was. by in. to emperor emperor in, the was, by with emperor, the emperor king the toig�� the. His was the of


In [ ]:
# prompt: give the context to the gpt which is defined abot

# Assuming 'context' is already defined from the previous code as:
# context = "Context:\n"
# for i in range(top_k):
#     context = context + response.source_nodes[i].text + "\n\n"

# Now, to give this context to the GPT model, you'll need to incorporate it into the prompt:
input_text = context + "Tell me about the story of King Parikshit"

# Generate text using the fine-tuned model
result = generator(input_text, max_length=100, num_return_sequences=1)

# Print the generated text
print(result[0]['generated_text'])


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


ValueError: Input length of input_ids is 100, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 

In [ ]:
!cd /content/bhagavatam_model

In [ ]:
!huggingface-cli repo create bhagavatham_model


git version 2.34.1
git-lfs/3.0.2 (GitHub; linux amd64; go 1.18.1)

You are about to create shivasathyanarayana/bhagavatham_model
Proceed? [Y/n] y
 (Request ID: Root=1-66b7895a-11a852d825b4da9222a6179d;e699a92d-f799-4e39-b695-9cb936b3e374)

403 Forbidden: You don't have the rights to create a model under the namespace "shivasathyanarayana".
Cannot access content at: https://huggingface.co/api/repos/create.
If you are trying to create or update content,make sure you have a token with the `write` role.
{"error":"You don't have the rights to create a model under the namespace \"shivasathyanarayana\""}


### Use LLM

In [ ]:
# prompt (no context)
intstructions_string = f""" Think you are a spiritual master of ISKCON and you are guiding some people who are facing with some problems and some questions.

Please respond to the following comment.
"""
prompt_template = lambda comment: f'''[INST] {intstructions_string} \n{comment} \n[/INST]'''

In [ ]:
comment = "Who is god?"

prompt = prompt_template(comment)
print(prompt)

[INST]  Think you are a spiritual master of ISKCON and you are guiding some people who are facing with some problems and some questions.

Please respond to the following comment.
 
Who is god? 
[/INST]


In [ ]:
model.eval()

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1797: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cuda, whereas the model is on cpu. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cpu') before running `.generate()`.
  warnings.warn(


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [ ]:
# prompt (with context)
prompt_template_w_context = lambda context, comment: f"""[INST]you are isko leader you are assigned to clarify the doubts based on bhagavatham.

{context}
Please respond to the following comment. Use the context above if it is helpful.

{comment}

[/INST]
"""

In [ ]:
# Assuming you have already loaded the model and tokenizer
#inputs = tokenizer(input_text, return_tensors="pt")

# Move inputs to the same device as the model (in this case, GPU)
inputs = inputs.to("cuda")

# Generate output with the model
outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=280)

# Decode the generated output
print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
